# CountVectorizer Models — split first, then oversample

This notebook mirrors the OG models but applies RandomOverSampler on the training set only (after train/test split).


In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from imblearn.over_sampling import RandomOverSampler

In [2]:
# Load data
df = pd.read_csv(
    "https://raw.githubusercontent.com/anoop-vs/nlp-climate-change/refs/heads/main/main_data.csv"
)
df.head()

,Unnamed: 0,content,label
0,0,Researchers use deep learning to simulate chlo...,Positive
1,1,Why is our @Conservatives government so evil?\...,Negative
2,2,"Texas Oilfield Waste Company Contributed $53,7...",Negative
3,3,Epic California snowpack is now the deepest it...,Negative
4,4,If #climatechange is real and not a hoax why d...,Negative


In [3]:
# NLTK prerequisites
nltk.download("stopwords")
nltk.download("wordnet")
nltk.download("omw-1.4")

[nltk_data] Downloading package stopwords to /Users/nafis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/nafis/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/nafis/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
# Preprocess and split
def preprocess(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    text = text.lower()
    words = text.split()
    sw = set(stopwords.words("english"))
    words = [w for w in words if w not in sw]
    stemmer = PorterStemmer()
    words = [stemmer.stem(w) for w in words]
    lemmatizer = WordNetLemmatizer()
    words = [lemmatizer.lemmatize(w) for w in words]
    return " ".join(words)


df["content"] = df["content"].apply(preprocess)
X = df["content"]
y = df["label"]
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [5]:
# Vectorize (CountVectorizer)
count_vectorizer = CountVectorizer()
X_train_count = count_vectorizer.fit_transform(X_train)
X_test_count = count_vectorizer.transform(X_test)

In [6]:
# Oversample training set only
oversampler = RandomOverSampler(random_state=42)
X_train_res, y_train_res = oversampler.fit_resample(X_train_count, y_train)
X_train_res.shape, X_test_count.shape

((4416, 12100), (882, 12100))

## Models


In [7]:
# Logistic Regression
logreg = LogisticRegression()
logreg.fit(X_train_res, y_train_res)
y_pred = logreg.predict(X_test_count)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.72      0.74      0.73       154
     Neutral       0.81      0.82      0.81       378
    Positive       0.84      0.81      0.82       350

    accuracy                           0.80       882
   macro avg       0.79      0.79      0.79       882
weighted avg       0.80      0.80      0.80       882



/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# Random Forest
rfc = RandomForestClassifier()
rfc.fit(X_train_res, y_train_res)
y_pred = rfc.predict(X_test_count)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.86      0.73      0.79       154
     Neutral       0.81      0.87      0.84       378
    Positive       0.83      0.83      0.83       350

    accuracy                           0.83       882
   macro avg       0.84      0.81      0.82       882
weighted avg       0.83      0.83      0.83       882



In [9]:
# Linear SVM
svm = SVC(kernel="linear", C=10, gamma=10)
svm.fit(X_train_res, y_train_res)
y_pred = svm.predict(X_test_count)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.73      0.78      0.75       154
     Neutral       0.84      0.85      0.84       378
    Positive       0.87      0.82      0.84       350

    accuracy                           0.83       882
   macro avg       0.81      0.82      0.81       882
weighted avg       0.83      0.83      0.83       882



In [10]:
# Decision Tree (OG-style params where applicable)
dt = DecisionTreeClassifier(
    max_depth=None, max_features="auto", min_samples_leaf=1, min_samples_split=2
)
dt.fit(X_train_res, y_train_res)
y_pred = dt.predict(X_test_count)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.62      0.54      0.58       154
     Neutral       0.75      0.72      0.73       378
    Positive       0.71      0.77      0.74       350

    accuracy                           0.71       882
   macro avg       0.69      0.68      0.68       882
weighted avg       0.71      0.71      0.71       882



/Users/nafis/code/python/climate change sentiment analysis/.venv311/lib/python3.11/site-packages/sklearn/tree/_classes.py:298: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'`.
  warnings.warn(


In [11]:
# Multinomial Naive Bayes (valid for non-negative sparse counts)
nb = MultinomialNB()
nb.fit(X_train_res, y_train_res)
y_pred = nb.predict(X_test_count)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

    Negative       0.54      0.72      0.62       154
     Neutral       0.82      0.73      0.77       378
    Positive       0.78      0.76      0.77       350

    accuracy                           0.74       882
   macro avg       0.71      0.74      0.72       882
weighted avg       0.76      0.74      0.74       882

